In [ ]:
import pandas as pd
dfmeta = pd.read_csv("./experiments/meta.csv")
dfruns = pd.read_csv("./experiments/runs.csv")
dfdata = pd.read_csv("./experiments/data.csv")

settings = ["2-S-S-IP10", 
            "2-S-S-IP50",
            "2-S-S-IP100"]

def get_setting_meta(setting):
    return dfmeta[ dfmeta["settingname"] == setting ].iloc[0]

In [ ]:
print("setting\t\truntime\t\titercount")
print("-"*50)
for setting in dfmeta["settingname"]:
    selected = dfruns[dfruns["run"] == setting]
    runtime = (selected["t_neighbours"].sum() + selected["t_rcs_sum"].sum())/1000
    itercount  = selected.shape[0]
    if len(setting) >= 7: # some ad-hoc code for nicer formatting
        print("%s\t%.2f\t\t%s" % (setting, runtime, itercount))
    else:
        print("%s\t\t%.2f\t\t%s" % (setting, runtime, itercount))

In [ ]:
import matplotlib.pyplot as plt
from mlcv import render_stats

fig, allaxes = plt.subplots(3,len(settings), figsize=(18,8))
for axes, setting in zip(allaxes.T, settings):
    selected = dfruns[dfruns["run"] == setting]
    _axes = (axes[0], None, None, axes[1], axes[2], None)
    render_stats(selected, axes=_axes)
fig.tight_layout()

In [ ]:
from mlcv import render_pcl
import numpy as np

# figure out distinct datasets
# compare settings on distinct datasets
dindices = sorted({ get_setting_meta(setting)["didx"] for setting in settings })
rows, cols = len(dindices), len(settings)+1

fig = plt.figure(figsize=(20,12))
for axidxrow, didx in enumerate(dindices):
    selected = dfdata[dfdata["didx"]==didx]
    gt =       selected["gt"].to_numpy()
    data = selected[["x","y","z"]].to_numpy()
    ax = fig.add_subplot(rows, cols, axidxrow*cols+1, projection="3d")
    ax.set_title("ground truth (ds %d)" % didx)
    render_pcl(data, gt, ax=ax)
    
for sidx, setting in enumerate(settings):
    selected_setting = get_setting_meta(setting)
    didx = selected_setting["didx"]
    selected_data = dfdata[ dfdata["didx"] == didx]
    data = selected_data[["x", "y", "z"]].to_numpy()
    part = selected_data[setting].to_numpy()
    
    axidxrow = np.where(dindices==didx)[0][0]
    axidxcol = sidx+1
    ax = fig.add_subplot(rows, cols, axidxrow*cols+axidxcol+1, projection='3d')
    ax.set_title("predicted (run '%s')" % setting )
    render_pcl(data, part, ax=ax)
    